In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [3]:
dem_df = pd.read_csv('../data/post_dem_candidates_with_tweet_topics2.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
rep_df = pd.read_csv('../data/post_rep_candidates_with_tweet_topics2.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])


# dem_df.loc[(dem_df['num_tweets'] > 0) & (dem_df['sentiment'].isna()), 'num_tweets'] = 0
# print(len(dem_df[dem_df['sentiment'].isna()]))
# dem_df.to_csv('../data/post_dem_candidates_with_tweet_topics.csv', index=False)

# rep_df.loc[(rep_df['num_tweets'] > 0) & (rep_df['sentiment'].isna()), 'num_tweets'] = 0
# print(len(rep_df[rep_df['sentiment'].isna()]))
# rep_df.to_csv('../data/post_rep_candidates_with_tweet_topics.csv', index=False)


sa_all_tweets = pd.read_csv('../data/tweet_data/sa_all_tweets_post.csv', index_col=False)

print(str(len(dem_df[(dem_df['won_primary'] == "Yes") & (dem_df['num_tweets'] >= 10)])) + " : " + str(len(dem_df[dem_df['won_primary'] == "Yes"])))
print(str(len(dem_df[(dem_df['won_primary'] == "No") & (dem_df['num_tweets'] >= 10)])) + " : " + str(len(dem_df[dem_df['won_primary'] == "No"])))

# Add these lines if you only want to include candidates who have tweeted 10+ times
dem_df = dem_df[dem_df['num_tweets'] >= 5]
rep_df = rep_df[rep_df['num_tweets'] >= 5]

# all_tweets_no_topics = pd.read_csv('../data/tweet_data/all_tweets_cleaned.csv', index_col=False)
# all_tweets_topics = pd.read_csv('../data/tweet_data/tweets_with_topics.csv', index_col=False)

rep_df.columns


188 : 202
396 : 543


Index(['candidate', 'twitter_handle', 'twitter_handle2', 'state', 'district',
       'office_type', 'race_type', 'race_primary_election_date',
       'primary_status', 'primary_runoff_status', 'general_status',
       'primary_pctg', 'won_primary', 'rep_party_support', 'trump_endorsed',
       'bannon_endorsed', 'great_america_endorsed', 'nra_endorsed',
       'right_to_life_endorsed', 'susan_b_anthony_endorsed',
       'club_for_growth_endorsed', 'koch_support', 'house_freedom_support',
       'tea_party_endorsed', 'main_street_endorsed', 'chamber_endorsed',
       'no_labels_support', 'has_twitter', 'num_endorsements',
       'gun_control_tweet_count', 'health_care_tweet_count',
       'abortion_tweet_count', 'immigration_tweet_count',
       'kavanaugh_tweet_count', 'education_tweet_count', 'jobs_tweet_count',
       'tax_tweet_count', 'fake_news_tweet_count', 'environment_tweet_count',
       'sexual_assault_tweet_count', 'gender_tweet_count', 'women_tweet_count',
       'lgbt_twee

In [21]:
topics = ['gun_control', 'health_care',
       'abortion', 'immigration', 'kavanaugh', 'jobs'
          , 'freedom', 'trump', 'gender']

# Encode data

In [22]:
drop_cols = ['primary_status', 'general_status', 'primary_runoff_status', 'twitter_handle', 'twitter_handle2']

encode_dem_cols = ['won_primary', 'candidate', 'veteran',
    'lgbtq', 'elected_official', 'self_funder', 'stem', 'obama_alum', 'dem_party_support', 'emily_endorsed',
    'gun_sense_candidate', 'biden_endorsed', 'warren_endorsed', 'sanders_endorsed', 'our_revolution_endorsed',
    'justice_dems_endorsed', 'pccc_endorsed', 'indivisible_endorsed', 'wfp_endorsed', 'votevets_endorsed', 'no_labels_support']

encode_rep_cols = ['won_primary','candidate', 
    'rep_party_support', 'trump_endorsed', 'bannon_endorsed', 'great_america_endorsed', 'nra_endorsed',
    'right_to_life_endorsed', 'susan_b_anthony_endorsed', 'club_for_growth_endorsed', 'koch_support', 'house_freedom_support',
    'tea_party_endorsed', 'main_street_endorsed','chamber_endorsed', 'no_labels_support']

topic_count_col = ['{}_tweet_count'.format(t) for t in topics]
topic_sent_col = ['{}_sentiment'.format(t) for t in topics]

encode_dem_cols.extend(topic_sent_col)
encode_rep_cols.extend(topic_sent_col)
model_dem_cols = encode_dem_cols.copy()
model_dem_cols.extend(['num_endorsements', 'num_tweets', 'sentiment'])
# model_dem_cols.extend(['num_endorsements', 'num_tweets', 'count_negative', 'count_neutral', 'count_positive', 'sentiment'])
model_dem_cols.extend(topic_count_col)

model_rep_cols = encode_rep_cols.copy()
model_rep_cols.extend(['num_endorsements', 'num_tweets', 'sentiment'])
# model_rep_cols.extend(['num_endorsements', 'num_tweets', 'count_negative', 'count_neutral', 'count_positive', 'sentiment'])
model_rep_cols.extend(topic_count_col)

label_encoder = LabelEncoder()
state_map = {}
def label_encode_data(df, encode_cols):
    for col in encode_cols:
        print(col)
        df[col] = label_encoder.fit_transform(df[col].values.astype(str))
        le_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
        if col == 'state':
            state_map = le_mapping
            print(state_map)
        if col not in ['candidate', 'state', 'district', 'race_primary_election_date']:
            print(le_mapping)

    return df

dem_df_enc = label_encode_data(dem_df, encode_dem_cols)
rep_df_enc = label_encode_data(rep_df, encode_rep_cols)


won_primary
{'0': 0, '1': 1, '2': 2}
candidate
veteran
{'0': 0, '1': 1, '2': 2}
lgbtq
{'0': 0, '1': 1, '2': 2}
elected_official
{'0': 0, '1': 1, '2': 2}
self_funder
{'0': 0, '1': 1}
stem
{'0': 0, '1': 1, '2': 2}
obama_alum
{'0': 0, '1': 1, '2': 2}
dem_party_support
{'0': 0, '1': 1, '2': 2}
emily_endorsed
{'0': 0, '1': 1, '2': 2}
gun_sense_candidate
{'0': 0, '1': 1, '2': 2}
biden_endorsed
{'0': 0, '1': 1, '2': 2}
warren_endorsed
{'0': 0, '1': 1, '2': 2}
sanders_endorsed
{'0': 0, '1': 1, '2': 2}
our_revolution_endorsed
{'0': 0, '1': 1, '2': 2}
justice_dems_endorsed
{'0': 0, '1': 1, '2': 2}
pccc_endorsed
{'0': 0, '1': 1, '2': 2}
indivisible_endorsed
{'0': 0, '1': 1, '2': 2}
wfp_endorsed
{'0': 0, '1': 1, '2': 2}
votevets_endorsed
{'0': 0, '1': 1, '2': 2}
no_labels_support
{'0': 0, '1': 1, '2': 2}
gun_control_sentiment
{'0': 0, '1': 1, '10': 2, '100': 3, '101': 4, '102': 5, '103': 6, '104': 7, '105': 8, '106': 9, '107': 10, '108': 11, '109': 12, '11': 13, '110': 14, '111': 15, '112': 16, '1

# Democratic mixedlm

### health care

In [11]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['health_care_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


40
39
606
606
['candidate ', ' veteran ', ' lgbtq ', ' elected_official ', ' self_funder ', ' stem ', ' obama_alum ', ' dem_party_support ', ' emily_endorsed ', ' gun_sense_candidate ', ' biden_endorsed ', ' warren_endorsed ', ' sanders_endorsed ', ' our_revolution_endorsed ', ' justice_dems_endorsed ', ' pccc_endorsed ', ' indivisible_endorsed ', ' wfp_endorsed ', ' votevets_endorsed ', ' no_labels_support ', ' gun_control_sentiment ', ' health_care_sentiment ', ' abortion_sentiment ', ' immigration_sentiment ', ' kavanaugh_sentiment ', ' jobs_sentiment ', ' freedom_sentiment ', ' trump_sentiment ', ' num_endorsements ', ' num_tweets ', ' sentiment ', ' gun_control_tweet_count ', ' health_care_tweet_count ', ' abortion_tweet_count ', ' immigration_tweet_count ', ' kavanaugh_tweet_count ', ' jobs_tweet_count ', ' freedom_tweet_count ', ' trump_tweet_count']
Index(['won_primary', 'candidate', 'veteran', 'lgbtq', 'elected_official',
       'self_funder', 'stem', 'obama_alum', 'dem_party_

## trump

In [15]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['trump_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


num candidates:  531
598
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    won_primary
No. Observations:    537        Method:                REML       
No. Groups:          531        Scale:                 0.0970     
Min. group size:     1          Likelihood:            -501.7499  
Max. group size:     2          Converged:             Yes        
Mean group size:     1.0                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -0.694    0.275 -2.524 0.012 -1.233 -0.155
candidate               -0.000    0.000 -2.047 0.041 -0.000 -0.000
veteran                  0.069    0.056  1.227 0.220 -0.041  0.180
lgbtq                    0.001    0.080  0.016 0.987 -0.156  0.158
elected_official         0.028    0.055  0.518 0.605

## jobs

In [17]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['jobs_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


num candidates:  523
598
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    won_primary
No. Observations:    530        Method:                REML       
No. Groups:          523        Scale:                 0.0819     
Min. group size:     1          Likelihood:            -486.8861  
Max. group size:     2          Converged:             Yes        
Mean group size:     1.0                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -0.647    0.279 -2.316 0.021 -1.194 -0.099
candidate               -0.000    0.000 -1.944 0.052 -0.000  0.000
veteran                  0.057    0.056  1.022 0.307 -0.052  0.166
lgbtq                   -0.002    0.078 -0.023 0.982 -0.156  0.152
elected_official         0.022    0.052  0.424 0.671

## immigration

In [18]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['immigration_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


num candidates:  471
598
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    won_primary
No. Observations:    478        Method:                REML       
No. Groups:          471        Scale:                 0.1829     
Min. group size:     1          Likelihood:            -453.1962  
Max. group size:     2          Converged:             Yes        
Mean group size:     1.0                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -0.741    0.293 -2.528 0.011 -1.315 -0.166
candidate               -0.000    0.000 -1.362 0.173 -0.000  0.000
veteran                  0.045    0.062  0.726 0.468 -0.076  0.166
lgbtq                   -0.031    0.084 -0.363 0.717 -0.196  0.135
elected_official         0.056    0.057  0.989 0.323

## freedom

In [19]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['freedom_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


num candidates:  386
598
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    won_primary
No. Observations:    390        Method:                REML       
No. Groups:          386        Scale:                 0.0000     
Min. group size:     1          Likelihood:            -352.2126  
Max. group size:     2          Converged:             Yes        
Mean group size:     1.0                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -1.131    0.348 -3.252 0.001 -1.813 -0.449
candidate               -0.000    0.000 -1.760 0.078 -0.001  0.000
veteran                  0.068    0.065  1.049 0.294 -0.059  0.195
lgbtq                    0.033    0.090  0.369 0.712 -0.144  0.211
elected_official         0.039    0.058  0.673 0.501

## gender

In [23]:
dem_x_cols = " + ".join(model_dem_cols[1:])
dem_model_data = dem_df_enc[model_dem_cols] # includes candidate
dem_model_data_topic = dem_model_data[dem_model_data['gender_tweet_count'] > 0]

print("num candidates: ", len(dem_model_data_topic.candidate.unique()))
print(len(dem_model_data.candidate.unique()))

model = smf.mixedlm("won_primary ~ " + dem_x_cols, dem_model_data_topic, groups=dem_model_data_topic["candidate"]).fit()
# model.summary().tables[1].to_csv('dem_mixedlm_won_primary.csv')
print(model.summary())


num candidates:  276
598
              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    won_primary
No. Observations:    279        Method:                REML       
No. Groups:          276        Scale:                 0.1395     
Min. group size:     1          Likelihood:            -297.8281  
Max. group size:     2          Converged:             Yes        
Mean group size:     1.0                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -1.053    0.412 -2.558 0.011 -1.860 -0.246
candidate               -0.000    0.000 -1.538 0.124 -0.001  0.000
veteran                  0.079    0.076  1.048 0.295 -0.069  0.227
lgbtq                    0.040    0.113  0.353 0.724 -0.182  0.262
elected_official        -0.051    0.069 -0.728 0.467